In [1]:
import sys
sys.path.append('../../')
from data.raw_datas import gemini_prompt
import google.generativeai as genai
from dotenv import load_dotenv
from pathlib import Path
import json
import time
import os

load_dotenv()

# 제미나이 불러오기
api_key = os.getenv("GEMNI_API_KEY")
genai.configure(api_key=api_key)

model = genai.GenerativeModel('gemini-2.0-flash')


# telegram 필터링된 데이터 불러오기
filtered_telegram_data_path = "../data/filtered_telegram_data.json"
with open(filtered_telegram_data_path, 'r', encoding='utf-8') as f:
    filtered_telegram_data = json.load(f)


/Users/bongjoon/Desktop/project/ADA/sogood/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def ask_gemini(sector, data):
    prompt = gemini_prompt(sector, data)
    response = model.generate_content(prompt)
    response_filter = response.text[7:-3].strip()
    try:
        return json.loads(response_filter)
    except json.JSONDecodeError as e:
        print(f"JSON 파싱 오류: {e}")
        return None



In [3]:
datas = {}

for sector in filtered_telegram_data.keys():

    if sector not in datas:
        datas[sector] = {}
        
    for date in filtered_telegram_data[sector]:
        try:
            print(sector, date)
            datas[sector][date] = ask_gemini(sector,str(dict(filtered_telegram_data[sector][date])))
            time.sleep(2)
            
        except Exception as e:
            print(f"오류 발생: {type(e).__name__} - {str(e)}")

전선 2025-05-27
전선 2025-05-22
전선 2025-05-21
전선 2025-05-25
전선 2025-05-26
전선 2025-05-23
전선 2025-05-24
전선 2025-05-20
IT 2025-05-27
IT 2025-05-26
IT 2025-05-24
IT 2025-05-23
IT 2025-05-22
IT 2025-05-21
IT 2025-05-20
IT 2025-05-25
방산 2025-05-27
방산 2025-05-26
방산 2025-05-24
방산 2025-05-23
방산 2025-05-22
방산 2025-05-21
방산 2025-05-20
방산 2025-05-25
원전 2025-05-27
원전 2025-05-24
원전 2025-05-21
원전 2025-05-25
원전 2025-05-23
원전 2025-05-26
원전 2025-05-22
원전 2025-05-20
조선 2025-05-27
조선 2025-05-25
조선 2025-05-23
조선 2025-05-21
조선 2025-05-26
조선 2025-05-24
조선 2025-05-22
조선 2025-05-20
반도체 2025-05-27
반도체 2025-05-24
반도체 2025-05-22
반도체 2025-05-21
반도체 2025-05-25
반도체 2025-05-26
반도체 2025-05-23
반도체 2025-05-20
풍력 2025-05-27
풍력 2025-05-23
풍력 2025-05-22
풍력 2025-05-21
풍력 2025-05-25
풍력 2025-05-20
풍력 2025-05-26
풍력 2025-05-24
건설 2025-05-27
건설 2025-05-21
건설 2025-05-25
건설 2025-05-23
건설 2025-05-22
건설 2025-05-26
건설 2025-05-24
건설 2025-05-20
여행 2025-05-27
여행 2025-05-26
여행 2025-05-24
여행 2025-05-21
여행 2025-05-22
여행 2025-05-25
여행 2025-05-2

In [4]:
gemini_result_data_path = "../data/gemini_result_data.json"

with open(gemini_result_data_path, 'w', encoding='utf-8') as f:
    json.dump(datas, f, ensure_ascii=False, indent=2)
